In [1]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import os


from tensorflow.keras import datasets
from tensorflow.keras import layers

def MyModel(input_shape, num_classes) :
    inputs = layers.Input(shape = input_shape)

    net = layers.Conv2D(32, 3, 1, padding = 'SAME')(inputs)
    net = layers.Activation('relu')(net)
    net = layers.Conv2D(32, 3, 1, padding = 'SAME')(net)
    net = layers.Activation('relu')(net)
    net = layers.MaxPool2D((2,2))(net)
    net = layers.Dropout(0.25)(net)

    net = layers.Conv2D(64, 3, 1, padding = 'SAME')(net)
    net = layers.Activation('relu')(net)
    net = layers.MaxPool2D((2,2))(net)
    net = layers.Conv2D(128, 3, 1, padding = 'SAME')(net)
    net = layers.Activation('relu')(net)
    net = layers.MaxPool2D((2,2))(net)
    net = layers.Dropout(0.25)(net)


    # Fully Connected
    net = layers.Flatten()(net)
    net = layers.Dense(1024)(net)
    net = layers.Activation('relu')(net)
    net = layers.Dropout(0.25)(net)
    net = layers.Dense(num_classes)(net)
    net = layers.Activation('softmax')(net)

    model = tf.keras.Model(inputs = inputs, outputs = net, name = 'TestCNN')
    
    return model

In [2]:
train_data_paths = []
train_y = []

for index in range(53):
    paths = tf.io.matching_files('./Card_imgs/train_img/{}/*.jpg'.format(index))
    #paths = tf.io.matching_files('./Card_imgs/train_img/{}/*.jpg'.format(index))
    # len(paths) -> 각 이미지의 개수,  1 * 인덱스 -> 분류할 데이터
    '''
    for i in range(len(paths)):
        label_name.append(card)
    label_name = np.array(label_name)
    print(type(label_name[0]))
    train_y.append(label_name)    
    train_data_paths.append(paths)
    '''
    # len(paths) -> 각 이미지의 개수,  1 * 인덱스 -> 분류할 데이터
    label_num = np.ones(len(paths)) * index
    train_data_paths.append(paths)
    train_y.append(label_num)

train_data_paths_1d = tf.concat(train_data_paths, axis=0)
train_y_1d = np.concatenate(train_y)
print(train_data_paths_1d.shape); print(train_y_1d.shape)

(4803,)
(4803,)


In [3]:
def tf_read_image(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile, dtype = tf.float32)
    return image
def make_batch(batch_paths):
    batch_images = []
    for path in batch_paths:
        image = tf_read_image(path)
        image.numpy()
        image = tf.image.resize(image, [150, 100])
        batch_images.append(image)
    return tf.convert_to_tensor(batch_images)

train_x_1d = make_batch(train_data_paths_1d)

In [4]:
checkpoint_path = "C:/Users/user/Desktop/졸업작품/ModelCheckPoint"
#checkpoint_path = "C:\Users\염정혁\Desktop\파이썬 연습\졸작\ModelCheckPoint"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

num_classes = 53
input_shape = (150, 100, 3)
model = MyModel(input_shape, num_classes)
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

num_epochs = 10
batch_size = 32
hist = model.fit(train_x_1d, train_y_1d, batch_size = batch_size, shuffle = True, epochs = num_epochs)

Epoch 1/10
151/151 [==============================] - 65s 426ms/step - loss: 3.0701 - accuracy: 0.2844
Epoch 2/10
151/151 [==============================] - 65s 432ms/step - loss: 0.0580 - accuracy: 0.9903
Epoch 3/10
151/151 [==============================] - 67s 441ms/step - loss: 0.0298 - accuracy: 0.9928
Epoch 4/10
151/151 [==============================] - 65s 433ms/step - loss: 0.0199 - accuracy: 0.9947
Epoch 5/10
151/151 [==============================] - 66s 436ms/step - loss: 0.0152 - accuracy: 0.9957
Epoch 6/10
151/151 [==============================] - 66s 437ms/step - loss: 0.0155 - accuracy: 0.9954
Epoch 7/10
151/151 [==============================] - 66s 440ms/step - loss: 0.0087 - accuracy: 0.9969
Epoch 8/10
151/151 [==============================] - 66s 440ms/step - loss: 0.0113 - accuracy: 0.9961
Epoch 9/10
151/151 [==============================] - 67s 441ms/step - loss: 0.0129 - accuracy: 0.9956
Epoch 10/10
151/151 [==============================] - 68s 447ms/step - l

In [5]:
model.save('my_models.h5')